# **Loading libraries**



In [ ]:
%pip install gdown
%pip install tqdm scikit-learn
%pip install geopandas
%pip install geohash2
%pip install folium
%pip install python-geohash



import pandas as pd
import plotly.express as px
import gdown
import os
import plotly.graph_objects as go
import numpy as np
from scipy.stats import gaussian_kde
from google.colab import drive
from math import radians, sin, cos, sqrt, atan2
import geopandas as gpd
from shapely.geometry import Point
from sklearn.neighbors import BallTree
from tqdm import tqdm
import geohash2
from sklearn.cluster import KMeans
import json
import folium
from folium.plugins import MarkerCluster
from scipy.stats import chi2_contingency
import geohash as gh







In [ ]:
.drive.mount('/content/drive')


# **Loading the data**



---
downloading the dataset
---



In [ ]:
folder_id = '1O3w5OKnS__hzlL8kTSfGCUc_iX8XNjEN'
output_dir = 'Homework'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
print(f"Attempting to download content from folder ID: {folder_id} into {output_dir}")
try:
    gdown.download_folder(id=folder_id, output=output_dir, quiet=False, use_cookies=False)
    print(f"\nSuccessfully downloaded content to: /content/{output_dir}")
    print("You can now find the downloaded content in the 'downloaded_external_folder' directory in your Colab files browser.")
except Exception as e:
    print(f"\nAn error occurred during download: {e}")
    print("Please ensure the Google Drive folder is publicly accessible or shared with 'Anyone with the link can view'.")

stations_info=pd.read_csv("Homework/data/Capital_Bikeshare_Locations.csv")
#
# Load tabular data
weather_df = pd.read_csv("Homework/data/Washington,DC,USA 2024-01-01 to 2024-12-31.csv")
trips_df = pd.read_parquet('Homework/data/daily-rent.parquet')

# Load spatial parking zones
parking_zones_gdf = gpd.read_file('Homework/data/Residential_and_Visitor_Parking_Zones.geojson')

stations_df = pd.read_csv("Homework/data/Capital_Bikeshare_Locations.csv")
# Load spatial parking zones
parking_zones_gdf = gpd.read_file('Homework/data/Residential_and_Visitor_Parking_Zones.geojson')
Shuttle_Bus_Stops=pd.read_csv("Homework/data/Shuttle_Bus_Stops.csv")
Metro_Bus_Stops =pd.read_csv("Homework/data/Metro_Bus_Stops.csv")
#Loading Residential and Visitor Parking Zones
Residential_Visitor_Parking_Zones  = gpd.read_file('Homework/data/Residential_and_Visitor_Parking_Zones.geojson')






---
Downloading the combined and modified dataset (for ease of use )
---



In [ ]:
file_id = "114g7JYuZ00i864przAIJQYymib_5h6Qa"  # Replace with your actual file ID
output_file = "trips_df.csv"  # You can change the output file name

gdown.download(id=file_id, output=output_file, quiet=False)
trips_df = pd.read_csv(output_file)
print(f"File downloaded to {output_file}")


In [ ]:
trips_df.head(2)

# **Preprocessing , Cleaning & inspecting the data**


There is a problem with missing start/id , almost 20% of the data are nulls so we must find a way to fill these up

**spatial join**


---


using lang and lati we can match it to the nearest station and then assign this id

In [ ]:
trips_df = trips_df.dropna(subset=['end_lat', 'end_lng'])

trips_df_cleaned=trips_df.drop_duplicates()
trips_df_cleaned.isna().sum()

In [ ]:
# EPSG:4326 = lat/lon
trips_gdf = gpd.GeoDataFrame(
    trips_df,
    geometry=gpd.points_from_xy(trips_df['start_lng'], trips_df['start_lat']),
    crs='EPSG:4326'
)

stations_gdf = gpd.GeoDataFrame(
    stations_df,
    geometry=gpd.points_from_xy(stations_df['LONGITUDE'], stations_df['LATITUDE']),
    crs='EPSG:4326'
)


In [ ]:
# Find nearest station to each ride
trips_with_nearest_station = gpd.sjoin_nearest(
    trips_gdf, stations_gdf[['STATION_ID', 'geometry']],
    how="left", distance_col="distance"
)

# Now we fill missing station_id with nearest one
trips_df['start_station_id'] = trips_df['start_station_id'].fillna(
    trips_with_nearest_station['STATION_ID']
)
# Creating a mapping from STATION_ID to STATION_NAME
id_to_name = stations_df.set_index('STATION_ID')['NAME'].to_dict()

# Fill in missing start_station_name using start_station_id
trips_df['start_station_name'] = trips_df['start_station_name'].fillna(
    trips_df['start_station_id'].map(id_to_name)
)
trips_df_cleaned=trips_df.drop_duplicates()
trips_df_cleaned.isna().sum()

Repeating the process to end id and name

In [ ]:
trips_gdf_end = gpd.GeoDataFrame(
    trips_df,
    geometry=gpd.points_from_xy(trips_df['end_lng'], trips_df['end_lat']),
    crs='EPSG:4326'
)

trips_with_nearest_end_station = gpd.sjoin_nearest(
    trips_gdf_end, stations_gdf[['STATION_ID', 'geometry']],
    how="left", distance_col="end_distance"
)

trips_df['end_station_id'] = trips_df['end_station_id'].fillna(
    trips_with_nearest_end_station['STATION_ID']
)
trips_df['end_station_name'] = trips_df['end_station_name'].fillna(
    trips_df['end_station_id'].map(id_to_name)
)
trips_df=trips_df.drop_duplicates()
trips_df.isna().sum()

we will continue inspecting the rest of the data

In [ ]:
stations_df=stations_df.drop_duplicates()
stations_df.isna().sum()  # we dont need to drop null values

In [ ]:
weather_df=weather_df.drop_duplicates()
weather_df.isna().sum()

In [ ]:
parking_zones_gdf.head(2)

In [ ]:
parking_zones_gdf=parking_zones_gdf.drop_duplicates()
parking_zones_gdf.isna().sum()

In [ ]:
parking_zones_gdf = parking_zones_gdf.drop(columns=['CREATOR', 'CREATED','EDITOR','EDITED'])


In [ ]:
parking_zones_gdf=parking_zones_gdf.drop_duplicates()
parking_zones_gdf.isna().sum()

---
**The outside WDC problem :**

In [ ]:
# The bounding box method
DC_LAT_MIN = 38.7916
DC_LAT_MAX = 38.9955
DC_LNG_MIN = -77.1198
DC_LNG_MAX = -76.9094
# Filtering  Points Outside the Bounding Box
out_of_bounds_start = ~(
    (trips_df['start_lat'].between(DC_LAT_MIN, DC_LAT_MAX)) &
    (trips_df['start_lng'].between(DC_LNG_MIN, DC_LNG_MAX))
)

out_of_bounds_end = ~(
    (trips_df['end_lat'].between(DC_LAT_MIN, DC_LAT_MAX)) &
    (trips_df['end_lng'].between(DC_LNG_MIN, DC_LNG_MAX))
)

# Combine both to detect any trip with at least one bad coordinate
outlier_mask = out_of_bounds_start | out_of_bounds_end
outliers = trips_df[outlier_mask]

# Inspect the Outliers
print(f"Number of outlier trips: {len(outliers)}")
outliers[['ride_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng']].head()
# Total number of trips in the dataset
total_trips = len(trips_df)

# Number of outliers detected
num_outliers = len(outliers)

# Calculate the percentage of outliers
percentage_outliers = (num_outliers / total_trips) * 100

print(f"Percentage of outlier trips: {percentage_outliers:.2f}%")


In [ ]:
# we will drop them
trips_df = trips_df[~outlier_mask].reset_index(drop=True)


In [ ]:
# checking ride_id
print("Duplicate ride_ids:", trips_df['ride_id'].duplicated().sum())
print("Missing ride_ids:", trips_df['ride_id'].isna().sum())


*there an issue with dublicated ride_id so we will only keep the first occurrence*

In [ ]:
# Keep first occurrence or drop based on your context:
trips_df = trips_df.drop_duplicates(subset='ride_id', keep='first')

In [ ]:
print("Null times:", trips_df[['started_at', 'ended_at']].isna().sum())
print("Negative durations:", (trips_df['ended_at'] < trips_df['started_at']).sum())


In [ ]:
print("Missing start station:", trips_df['start_station_id'].isna().sum())
print("Missing end station:", trips_df['end_station_id'].isna().sum())


In [ ]:
zero_coords = trips_df[(trips_df['start_lat'] == 0) | (trips_df['start_lng'] == 0)]
print("Zero coordinates:", len(zero_coords))

In [ ]:
# checkign if rideable_type and member_casual has weird values
print("Ride types:", trips_df['rideable_type'].unique())
print("Member types:", trips_df['member_casual'].unique())


In [ ]:
weather_df.head(1)

In [ ]:
# first we make sure all the dates are in the same format (by checking the length)
datetime_lengths = weather_df["datetime"].astype(str).apply(len)
print(datetime_lengths.value_counts())
weather_df["date"] = pd.to_datetime(weather_df["datetime"])
print(weather_df["date"].dtype)


In [ ]:
trips_df["start_time"] = pd.to_datetime(trips_df["started_at"])
trips_df["end_time"] = pd.to_datetime(trips_df["ended_at"])
# ensuring that CRS is EPSG:4326
if parking_zones_gdf.crs != "EPSG:4326":
    parking_zones_gdf = parking_zones_gdf.to_crs("EPSG:4326")
# Spatial Join to Map Stations to Parking Zones
# Spatial join: add zone info to each station
stations_with_zone = gpd.sjoin(
    stations_gdf,
    parking_zones_gdf[["NAME", "geometry"]],
    how="left",
    predicate="within"
)
# Rename column for clarity
stations_with_zone = stations_with_zone.rename(columns={"zone_name": "residential_zone"})
# Joining Weather Data
# Extract date from start_time for weather join
trips_df["date"] = trips_df["start_time"].dt.date
weather_df["date"] = weather_df["date"].dt.date

# Join weather by date
trips_df = trips_df.merge(weather_df, on="date", how="left")



In [ ]:
trips_df[['start_station_id', 'end_station_id', 'start_station_name', 'end_station_name']].isnull().sum()


In [ ]:
trips_df.head(2)



---

# **Feature engineering**

---




---
B1
---
استخلاص المكونات الزمنية من الرحلات


In [ ]:
# B1

# From started_at
trips_df['start_year'] = trips_df['started_at'].dt.year
trips_df['start_month'] = trips_df['started_at'].dt.month
trips_df['start_day_num'] = trips_df['started_at'].dt.day
trips_df['start_day_name'] = trips_df['started_at'].dt.day_name()

# From ended_at
trips_df['end_year'] = trips_df['ended_at'].dt.year
trips_df['end_month'] = trips_df['ended_at'].dt.month
trips_df['end_day_num'] = trips_df['ended_at'].dt.day
trips_df['end_day_name'] = trips_df['ended_at'].dt.day_name()
trips_df.head(2)



---
B2
---
حساب مدة الرحلة


In [ ]:
trips_df['trip_duration_minutes'] = (trips_df['end_time'] - trips_df['start_time']).dt.total_seconds() / 60
trips_df['trip_duration_minutes']=trips_df['trip_duration_minutes'].round(2)
trips_df['trip_duration_minutes'].head(3)

**The trip_duration_minutes problem**

In [ ]:
trips_df['trip_duration_minutes'].describe()

*we can clearly see that there is a problem with the tripd_durations, the min is a negative value and that is not right*

In [ ]:
# Show trips with negative or 0 duration
invalid_durations = trips_df[trips_df['trip_duration_minutes'] <= 0]
print(f"Invalid rows: {len(invalid_durations)}")
invalid_durations[['ride_id', 'started_at', 'ended_at', 'trip_duration_minutes']].head()


In [ ]:
# Filter only valid trips
trips_df = trips_df[trips_df['trip_duration_minutes'] > 0]
trips_df['trip_duration_minutes'].describe()



---
B3
---
حساب تكلفة الرحلة  


In [ ]:
trips_df['member_casual'].value_counts()

In [ ]:
trips_df['rideable_type'].value_counts()

In [ ]:
# Initialize base cost
# Start with 0 cost
trips_df['trip_cost'] = 0.0

# Define fixed costs
trips_df.loc[trips_df['member_casual'] == 'member', 'trip_cost'] = 3.95
trips_df.loc[trips_df['member_casual'] == 'casual', 'trip_cost'] = 1.00

# Add extra cost for duration
# for members :
# Create condition for member rides longer than 45 mins
cond_member_extra = (trips_df['member_casual'] == 'member') & (trips_df['trip_duration_minutes'] > 45)

# Electric bike extra for members
trips_df.loc[cond_member_extra & (trips_df['rideable_type'] == 'electric_bike'), 'trip_cost'] += \
    (trips_df['trip_duration_minutes'] - 45) * 0.10

# Classic bike extra for members
trips_df.loc[cond_member_extra & (trips_df['rideable_type'] == 'classic_bike'), 'trip_cost'] += \
    (trips_df['trip_duration_minutes'] - 45) * 0.05
# Electric bike for casuals
cond_casual_electric = (trips_df['member_casual'] == 'casual') & (trips_df['rideable_type'] == 'electric_bike')
trips_df.loc[cond_casual_electric, 'trip_cost'] += trips_df['trip_duration_minutes'] * 0.15

# Classic bike for casuals
cond_casual_classic = (trips_df['member_casual'] == 'casual') & (trips_df['rideable_type'] == 'classic_bike')
trips_df.loc[cond_casual_classic, 'trip_cost'] += trips_df['trip_duration_minutes'] * 0.05
# Add Central Business District (CBD) fee
# Preparaing your geometry points
# Create GeoDataFrame of start points
trips_df['start_point'] = trips_df.apply(lambda row: Point(row['start_lng'], row['start_lat']), axis=1)
trips_df['end_point'] = trips_df.apply(lambda row: Point(row['end_lng'], row['end_lat']), axis=1)
# #  Load CBD Polygon
CBD = gpd.read_file('Homework/data/DDOT_Central_Business_District.geojson')
CBD = CBD.to_crs(epsg=4326)  # Ensures it's in WGS 84


# Convert to GeoDataFrames with correct CRS
start_gdf = gpd.GeoDataFrame(trips_df, geometry='start_point', crs='EPSG:4326').to_crs('EPSG:6933')
end_gdf = gpd.GeoDataFrame(trips_df, geometry='end_point', crs='EPSG:4326').to_crs('EPSG:6933')

# Load CBD polygon and project to EPSG:6933
# CBD = gpd.read_file('Homework/data/DDOT_Central_Business_District.geojson')
# CBD = CBD.to_crs(epsg=6933)
# cbd_polygon = CBD.geometry.unary_union  # Get full boundary

In [ ]:
# Load CBD polygon and project to EPSG:6933
CBD = gpd.read_file('Homework/data/DDOT_Central_Business_District.geojson')
CBD = CBD.to_crs(epsg=6933)
cbd_polygon = CBD.geometry.unary_union  # Get full boundary
CBD

In [ ]:
# Check spatial containment in EPSG:6933
trips_df['start_in_cbd'] = start_gdf['start_point'].apply(lambda point: point.within(cbd_polygon))
trips_df['end_in_cbd'] = end_gdf['end_point'].apply(lambda point: point.within(cbd_polygon))

# Final condition and cost update
trips_df['in_cbd'] = trips_df['start_in_cbd'] | trips_df['end_in_cbd']
trips_df.loc[trips_df['in_cbd'], 'trip_cost'] += 0.5
trips_df['trip_cost'].head()

In [ ]:
trips_df['trip_cost'].describe()

*we can see a clear issue in the data ,  and super high values (4.3 mil in the max ) and std is very high (4837.62) , so we must identify this outliers and deal with them*

In [ ]:
# High-cost trips
high_cost = trips_df[trips_df['trip_cost'] > 1000].copy()
print(high_cost[['ride_id', 'trip_duration_minutes', 'rideable_type', 'member_casual', 'trip_cost']])

# Negative-cost trips
neg_cost = trips_df[trips_df['trip_cost'] < 0].copy()
# try to only print the len
print(neg_cost[['ride_id', 'trip_duration_minutes', 'rideable_type', 'member_casual', 'trip_cost']])


In [ ]:
# Total rows
total_rows = len(trips_df)
# Define thresholds
high_cost_threshold = 10000
negative_cost_threshold = 0

# Find outliers
high_cost_outliers = trips_df[trips_df['trip_cost'] > high_cost_threshold]
negative_cost_outliers = trips_df[trips_df['trip_cost'] < negative_cost_threshold]

# Count
num_high_cost = len(high_cost_outliers)
num_negative_cost = len(negative_cost_outliers)
total_outliers = num_high_cost + num_negative_cost

# Percentages
percent_high_cost = (num_high_cost / total_rows) * 100
percent_negative_cost = (num_negative_cost / total_rows) * 100
percent_total_outliers = (total_outliers / total_rows) * 100

print(f"High cost outliers: {num_high_cost} ({percent_high_cost:.2f}%)")
print(f"Negative cost outliers: {num_negative_cost} ({percent_negative_cost:.2f}%)")
print(f"Total outliers: {total_outliers} ({percent_total_outliers:.2f}%)")


since they make a very small amount of the data (0.0%) they can be classifed as false data and we can drop them

In [ ]:
# Drop outliers by reassigning the filtered DataFrame back to df
trips_df = trips_df[(trips_df['trip_cost'] <= high_cost_threshold) & (trips_df['trip_cost'] >= negative_cost_threshold)]
trips_df['trip_cost'].describe()


---
B4
---
تصنيف سعة المحطات  


In [ ]:
stations_df['CAPACITY'].describe()

In [ ]:
# Basic histogram using Plotly
fig = px.histogram(stations_df, x='CAPACITY', nbins=30, title='Distribution of Station Capacity')
fig.update_layout(xaxis_title='Capacity', yaxis_title='Count', bargap=0.1)
fig.show(config={'staticPlot': True})


*Choosing the right threshold*

In [ ]:
# Drop NaNs
capacity_data = stations_df['CAPACITY'].dropna()
# Histogram
hist_data = go.Histogram(x=capacity_data, nbinsx=30, name='Histogram', opacity=0.6)
# Density Curve
kde = gaussian_kde(capacity_data)
x_vals = np.linspace(capacity_data.min(), capacity_data.max(), 1000)
kde_data = go.Scatter(x=x_vals, y=kde(x_vals) * len(capacity_data) * (x_vals[1] - x_vals[0]),
                      mode='lines', name='KDE Curve')

# Plot both
fig = go.Figure(data=[hist_data, kde_data])
fig.update_layout(title='Capacity Distribution with KDE',
                  xaxis_title='Capacity', yaxis_title='Count')
# Example thresholds
low_thresh = stations_df['CAPACITY'].quantile(0.30)
high_thresh = stations_df['CAPACITY'].quantile(0.66)
print(low_thresh,high_thresh)
fig.add_vline(x=low_thresh, line_dash="dash", line_color="green", annotation_text="Small/Average")
fig.add_vline(x=high_thresh, line_dash="dash", line_color="red", annotation_text="Average/Large")

fig.show(config={'staticPlot': True})


method 1 : using quantiles

In [ ]:
# Calculate the thresholds
low_thresh = stations_df['CAPACITY'].quantile(0.33)
high_thresh = stations_df['CAPACITY'].quantile(0.66)

def classify_capacity(cap):
    if cap <= low_thresh:
        return 'Small'
    elif cap <= high_thresh:
        return 'Average'
    else:
        return 'Large'

stations_df['STATION_SIZE'] = stations_df['CAPACITY'].apply(classify_capacity)
stations_df['STATION_SIZE'].value_counts()



method 2 : based on domain knowledge


In [ ]:
def classify_capacity(cap):
    if cap <= 12:
        return 'Small'
    elif cap <= 29:
        return 'Average'
    else:
        return 'Large'

stations_df['STATION_SIZE'] = stations_df['CAPACITY'].apply(classify_capacity)
stations_df['STATION_SIZE'].value_counts()
print(stations_df['STATION_SIZE'].value_counts())


combine with trips df

try1 : using the station_id

In [ ]:
# Step 2: Create a simplified DataFrame for merging
station_size_map = stations_df[['STATION_ID', 'STATION_SIZE']].copy()
# Step 3: Merge for start_station_size
trips_df = trips_df.merge(
    station_size_map.rename(columns={
        'STATION_ID': 'start_station_id',
        'STATION_SIZE': 'start_station_size'
    }),
    on='start_station_id',
    how='left'
)

# Step 4: Merge for end_station_size
trips_df = trips_df.merge(
    station_size_map.rename(columns={
        'STATION_ID': 'end_station_id',
        'STATION_SIZE': 'end_station_size'
    }),
    on='end_station_id',
    how='left'
)


In [ ]:
print("Missing start_station_size:", trips_df['start_station_size'].isna().sum())
print("Missing end_station_size:", trips_df['end_station_size'].isna().sum())


In [ ]:
# What kind of start_station_id had no match?
print(trips_df[trips_df['start_station_size'].isna()][['start_station_id']].drop_duplicates().head(10))

# Same for end_station
print(trips_df[trips_df['end_station_size'].isna()][['end_station_id']].drop_duplicates().head(10))


try2 : with names

In [ ]:
stations_df['NAME'] = stations_df['NAME'].str.strip().str.lower()
trips_df['start_station_name'] = trips_df['start_station_name'].str.strip().str.lower()
trips_df['end_station_name'] = trips_df['end_station_name'].str.strip().str.lower()

# Map start station size using name
station_size_name_map = stations_df[['NAME', 'STATION_SIZE']]

trips_df = trips_df.merge(
    station_size_name_map.rename(columns={'NAME': 'start_station_name', 'STATION_SIZE': 'start_station_size_name'}),
    on='start_station_name',
    how='left'
)

# Same for end station
trips_df = trips_df.merge(
    station_size_name_map.rename(columns={'NAME': 'end_station_name', 'STATION_SIZE': 'end_station_size_name'}),
    on='end_station_name',
    how='left'
)


In [ ]:
trips_df['end_station_size'].value_counts()

In [ ]:
trips_df['start_station_size'] = trips_df['start_station_size_name']
trips_df['end_station_size'] = trips_df['end_station_size_name']

# Then drop the temp columns
trips_df.drop(columns=['start_station_size_name', 'end_station_size_name'], inplace=True)


In [ ]:
print(trips_df[['start_station_size', 'end_station_size']].isna().sum())


using names was better but we still have some null values

In [ ]:

fig = px.histogram(stations_df, x='CAPACITY', nbins=30, title='Station Capacity Distribution')
fig.add_vline(x=15, line_dash="dash", line_color="green", annotation_text="Small/Average")
fig.add_vline(x=25, line_dash="dash", line_color="red", annotation_text="Average/Large")

fig.show(config={'staticPlot': True})


---
B5
---
حساب المسافة إلى أقرب محطة نقل عام



In [ ]:
Shuttle_Bus_Stops.isna().sum()


In [ ]:
Metro_Bus_Stops['BSTP_LAT'].isna().sum()


Approaches


---


| Approach                    | Time Complexity | Vectorized | Fast    |
| --------------------------- | --------------- | ---------- | ------- |
| Brute Force (Your original) | O(N × M)        | ❌ No       | 🐌 Slow |
| BallTree (New)              | O(N log M)      | ✅ Yes      | ⚡ Fast  |


Project all your coordinates to EPSG:6933


In [ ]:
# Create start and end point geometries
trips_df['start_point'] = trips_df.apply(lambda row: Point(row['start_lng'], row['start_lat']), axis=1)
trips_df['end_point'] = trips_df.apply(lambda row: Point(row['end_lng'], row['end_lat']), axis=1)

# Create GeoDataFrames
gdf_start = gpd.GeoDataFrame(trips_df, geometry='start_point', crs='EPSG:4326').to_crs(epsg=6933)
gdf_end = gpd.GeoDataFrame(trips_df, geometry='end_point', crs='EPSG:4326').to_crs(epsg=6933)

# Add x/y columns
trips_df['start_x'] = gdf_start.geometry.x
trips_df['start_y'] = gdf_start.geometry.y
trips_df['end_x'] = gdf_end.geometry.x
trips_df['end_y'] = gdf_end.geometry.y


# projecting   metro and shuttle station coordinates:

# Convert station lat/lng to projected coordinates
def project_coords(coords_list):
    gdf = gpd.GeoDataFrame(geometry=[Point(lon, lat) for lat, lon in coords_list], crs='EPSG:4326')
    gdf = gdf.to_crs(epsg=6933)
    return np.array([(geom.x, geom.y) for geom in gdf.geometry])
# coords
# Metro stop coordinates
metro_coords = Metro_Bus_Stops[['BSTP_LAT', 'BSTP_LON']].dropna().values

# Shuttle stop coordinates
shuttle_coords = Shuttle_Bus_Stops[['LATITUDE', 'LONGITUDE']].dropna().values

metro_coords_projected = project_coords(metro_coords)
shuttle_coords_projected = project_coords(shuttle_coords)


In [ ]:

def euclidean_tree_batch(source_df, stop_coords, x_col, y_col, batch_size=10000):
    tree = BallTree(stop_coords, metric='euclidean')

    distances = []
    n = len(source_df)
    tqdm.pandas(desc=f"Computing distances for {x_col}")

    for i in tqdm(range(0, n, batch_size), desc="Batch processing", unit="batch"):
        batch = source_df.iloc[i:i+batch_size]
        batch_points = batch[[x_col, y_col]].values

        dists, _ = tree.query(batch_points, k=1)
        distances.extend(dists.flatten().tolist())

    return distances


In [ ]:
# Start → Metro
trips_df['start_nearest_metro_distance'] = euclidean_tree_batch(
    trips_df, metro_coords_projected, 'start_x', 'start_y'
)

# End → Metro
trips_df['end_nearest_metro_distance'] = euclidean_tree_batch(
    trips_df, metro_coords_projected, 'end_x', 'end_y'
)

# Start → Shuttle
trips_df['start_nearest_shuttle_distance'] = euclidean_tree_batch(
    trips_df, shuttle_coords_projected, 'start_x', 'start_y'
)

# End → Shuttle
trips_df['end_nearest_shuttle_distance'] = euclidean_tree_batch(
    trips_df, shuttle_coords_projected, 'end_x', 'end_y'
)

# converting to from meters to km (our choice)
trips_df['start_nearest_metro_distance'] = trips_df['start_nearest_metro_distance'] / 1000
trips_df['end_nearest_metro_distance'] = trips_df['end_nearest_metro_distance'] / 1000
trips_df['start_nearest_shuttle_distance'] = trips_df['start_nearest_shuttle_distance'] / 1000
trips_df['end_nearest_shuttle_distance'] = trips_df['end_nearest_shuttle_distance'] / 1000


In [ ]:
trips_df[
    ['start_nearest_metro_distance',
     'end_nearest_metro_distance',
     'start_nearest_shuttle_distance',
     'end_nearest_shuttle_distance']
].describe()


In [ ]:
sampled_df = trips_df.sample(n=20000, random_state=50)

cols = ['start_nearest_metro_distance', 'end_nearest_metro_distance',
        'start_nearest_shuttle_distance', 'end_nearest_shuttle_distance']
for col in cols:
    fig = go.Figure(
        data=[go.Histogram(
            x=sampled_df[col],
            nbinsx=100,
            marker=dict(color='skyblue'),
            opacity=0.75
        )]
    )
    fig.update_layout(
        title=col,
        xaxis_title=col,
        yaxis_title='Count (Log Scale)',
        yaxis_type='log',
        bargap=0.1,
        width=800,
        height=400
    )
    fig.show(config={'staticPlot':True})

In [ ]:

# Re-define thresholds for clarity
start_nearest_metro_distance_thr = 1 # meters
end_nearest_metro_distance_thr = 1   # meters
start_nearest_shuttle_distance_thr = 9 # meters
end_nearest_shuttle_distance_thr = 9 # meters

# --- Step 1: Identify "far from" trips based on current thresholds ---
# Using the corrected end_nearest_shuttle_distance_thr for the end shuttle distance
far_metro_start_trips = trips_df[trips_df['start_nearest_metro_distance'] > start_nearest_metro_distance_thr]
far_metro_end_trips = trips_df[trips_df['end_nearest_metro_distance'] > end_nearest_metro_distance_thr]
far_shuttle_start_trips = trips_df[trips_df['start_nearest_shuttle_distance'] > start_nearest_shuttle_distance_thr]
far_shuttle_end_trips = trips_df[trips_df['end_nearest_shuttle_distance'] > end_nearest_shuttle_distance_thr]

# Combine all "far from transit" trips for a general map (for demonstration)
# Using a logical OR to get any trip that is far from ANY of these points
far_from_transit_trips = trips_df[
    (trips_df['start_nearest_metro_distance'] > start_nearest_metro_distance_thr) |
    (trips_df['end_nearest_metro_distance'] > end_nearest_metro_distance_thr) |
    (trips_df['start_nearest_shuttle_distance'] > start_nearest_shuttle_distance_thr) |
    (trips_df['end_nearest_shuttle_distance'] > end_nearest_shuttle_distance_thr)
].copy() # Use .copy() to avoid SettingWithCopyWarning

print(f"Number of trips far from metro (start): {len(far_metro_start_trips)}")
print(f"Number of trips far from metro (end): {len(far_metro_end_trips)}")
print(f"Number of trips far from shuttle (start): {len(far_shuttle_start_trips)}")
print(f"Number of trips far from shuttle (end): {len(far_shuttle_end_trips)}")
print(f"Total unique trips identified as 'far from transit': {len(far_from_transit_trips)}")


# --- Step 2: Create a Folium Map ---

# Get the approximate center of Washington D.C. for the map's initial view
# You can use the mean of your start/end lat/lngs or a known DC coordinate
dc_center_lat = trips_df['start_lat'].mean() # Or a more precise known center for DC
dc_center_lng = trips_df['start_lng'].mean() # Or a more precise known center for DC

# Create a base map
m = folium.Map(location=[dc_center_lat, dc_center_lng], zoom_start=12)

# Add markers for start points of trips identified as "far from transit"
# Due to the large number of potential points (40k), plotting individual markers for all
# might be slow or make the map unreadable.
# We'll plot a sample or use a MarkerCluster for better performance.
# Let's start by plotting a *sample* of these points if far_from_transit_trips is very large,
# or use MarkerCluster. For a first look, a small sample is good.

# If you have too many points, consider sampling for initial visualization
# Or, even better for density visualization, use MarkerCluster or HeatMap (if allowed for density, check project rules)
# Since you're using Plotly for charts and Folium for maps, heatmap should be fine.

# Let's just add a few to see the logic work, or use MarkerCluster for all:

# OPTION A: Plotting a limited sample (good for a quick check if map gets cluttered)
# sample_size = 1000 # Adjust as needed
# if len(far_from_transit_trips) > sample_size:
#     sample_to_plot = far_from_transit_trips.sample(sample_size, random_state=42)
# else:
#     sample_to_plot = far_from_transit_trips

# for idx, row in sample_to_plot.iterrows():
#     folium.CircleMarker(
#         location=[row['start_lat'], row['start_lng']],
#         radius=2, # Small radius
#         color='red',
#         fill=True,
#         fill_color='red',
#         fill_opacity=0.6,
#         tooltip=f"Start: {row['start_station_name']} (Far from Transit)"
#     ).add_to(m)

# OPTION B: Using MarkerCluster for better visualization of many points
from folium.plugins import MarkerCluster

marker_cluster_start = MarkerCluster().add_to(m)
marker_cluster_end = MarkerCluster().add_to(m)

# Add start points
for idx, row in far_from_transit_trips.iterrows():
    if pd.notnull(row['start_lat']) and pd.notnull(row['start_lng']):
        folium.CircleMarker(
            location=[row['start_lat'], row['start_lng']],
            radius=2,
            color='red', # Color for start points
            fill=True,
            fill_color='red',
            fill_opacity=0.6,
            tooltip=f"Start: {row['start_station_name']} (Far from Transit)"
        ).add_to(marker_cluster_start)

# Add end points (optional, you might want separate layers or colors if combining)
# For now, let's just show start points to avoid overwhelming the map.
# If you want to see end points, you could use a different color or a separate MarkerCluster
# for idx, row in far_from_transit_trips.iterrows():
#     if pd.notnull(row['end_lat']) and pd.notnull(row['end_lng']):
#         folium.CircleMarker(
#             location=[row['end_lat'], row['end_lng']],
#             radius=2,
#             color='blue', # Color for end points
#             fill=True,
#             fill_color='blue',
#             fill_opacity=0.6,
#             tooltip=f"End: {row['end_station_name']} (Far from Transit)"
#         ).add_to(marker_cluster_end)


# Save the map to an HTML file or display in a Jupyter Notebook
# m.save("far_from_transit_trips_map.html")
# In a Jupyter/IPython environment, you can also just display `m` directly
m

In [ ]:
# --- Define your new thresholds based on your histogram observations ---
# Example new thresholds (YOU WILL REPLACE THESE WITH YOUR OWN INSIGHTS)
new_metro_start_thr = 1  # meters (e.g., if you see a clear drop after 1.2km)
new_metro_end_thr = 1    # meters
new_shuttle_start_thr = 9 # meters (e.g., if you see a drop after 20km)
new_shuttle_end_thr = 9 # meters

# --- Create the new boolean features ---
trips_df['is_far_from_metro_start'] = trips_df['start_nearest_metro_distance'] > new_metro_start_thr
trips_df['is_far_from_metro_end'] = trips_df['end_nearest_metro_distance'] > new_metro_end_thr
trips_df['is_far_from_shuttle_start'] = trips_df['start_nearest_shuttle_distance'] > new_shuttle_start_thr
trips_df['is_far_from_shuttle_end'] = trips_df['end_nearest_shuttle_distance'] > new_shuttle_end_thr

# You can also create a combined flag for any "far from transit"
trips_df['is_far_from_any_transit'] = (
    trips_df['is_far_from_metro_start'] |
    trips_df['is_far_from_metro_end'] |
    trips_df['is_far_from_shuttle_start'] |
    trips_df['is_far_from_shuttle_end']
)

# Verify the counts of the new features
print("\nCounts for new 'far from' features:")
print(trips_df[['is_far_from_metro_start', 'is_far_from_metro_end',
                'is_far_from_shuttle_start', 'is_far_from_shuttle_end',
                'is_far_from_any_transit']].sum())

# Display the first few rows with the new columns to confirm
print("\nTrips DataFrame with new features:")
print(trips_df[['ride_id', 'start_nearest_metro_distance', 'is_far_from_metro_start',
                'start_nearest_shuttle_distance', 'is_far_from_shuttle_start',
                'is_far_from_any_transit']].head())

In [ ]:
trips_df['is_far_from_metro_start'].value_counts()

---
B6
---
تحديد ما إذا كانت الرحلة تمر بالمنطقة التجارية المركزية


In [ ]:
print(trips_df['start_point'].iloc[0], type(trips_df['start_point'].iloc[0]))
print(trips_df['end_point'].iloc[0], type(trips_df['end_point'].iloc[0]))
print(type(cbd_polygon))


In [ ]:
# STEP 0: Make sure the CBD polygon is projected correctly
CBD = CBD.to_crs(epsg=6933)
cbd_polygon = CBD.geometry.iloc[0]  # assuming a single polygon
# STEP 1: Create a GeoDataFrame from the trip points (start and end)
# start_gdf = gpd.GeoDataFrame(trips_df, geometry=trips_df['start_point'], crs="EPSG:4326")
# end_gdf   = gpd.GeoDataFrame(trips_df, geometry=trips_df['end_point'], crs="EPSG:4326")

# Rebuild the point geometries from lat/lng in EPSG:4326
start_gdf = gpd.GeoDataFrame(
    trips_df,
    geometry=gpd.points_from_xy(trips_df['start_lng'], trips_df['start_lat']),
    crs="EPSG:4326"
)

end_gdf = gpd.GeoDataFrame(
    trips_df,
    geometry=gpd.points_from_xy(trips_df['end_lng'], trips_df['end_lat']),
    crs="EPSG:4326"
)


# Project everything to EPSG:6933
CBD = CBD.to_crs(epsg=6933)
start_gdf = start_gdf.to_crs(epsg=6933)
end_gdf = end_gdf.to_crs(epsg=6933)

# CBD polygon (in same projection)
cbd_polygon = CBD.geometry.unary_union
# Check containment
trips_df['start_in_cbd'] = start_gdf['geometry'].apply(lambda pt: cbd_polygon.contains(pt))
trips_df['end_in_cbd']   = end_gdf['geometry'].apply(lambda pt: cbd_polygon.contains(pt))

# Final result
trips_df['in_cbd'] = trips_df['start_in_cbd'] | trips_df['end_in_cbd']
trips_df['in_cbd'].value_counts()

---
B7
---
حساب المسافة إلى المنطقة التجارية المركزية (CBD) وتصنيف القرب

In [ ]:
# --- Step 1: Compute the CBD centroid (already in EPSG:6933)
cbd_centroid = cbd_polygon.centroid  # geometry in meters (EPSG:6933)

# --- Step 2: Recreate end point GeoDataFrame and project to EPSG:6933
end_gdf = gpd.GeoDataFrame(
    trips_df,
    geometry=gpd.points_from_xy(trips_df['end_lng'], trips_df['end_lat']),
    crs="EPSG:4326"
).to_crs(epsg=6933)

# --- Step 3: Compute Euclidean distance in meters
trips_df['distance_to_cbd_m'] = end_gdf.geometry.distance(cbd_centroid)

# --- Step 4: Set distance to None where start AND end are in the CBD
mask = trips_df['start_in_cbd'] & trips_df['end_in_cbd']
trips_df.loc[mask, 'distance_to_cbd_m'] = None

# --- Step 5: Inspect result
trips_df['distance_to_cbd_m'].describe()




**Threasholding strategies**





kinda of an elbow method

In [ ]:
sampled_df = trips_df.sample(n=20000, random_state=50)

# Extract the data
data = sampled_df['distance_to_cbd_m'].dropna()

# Create histogram trace
hist = go.Histogram(
    x=data,
    nbinsx=100,
    name='Histogram',
    marker_color='lightblue',
    opacity=0.75
)

# Create KDE line (manual since Plotly doesn’t support KDE directly)
kde = gaussian_kde(data)
x_vals = np.linspace(data.min(), data.max(), 1000)
kde_vals = kde(x_vals) * len(data) * (x_vals[1] - x_vals[0])  # scale to match histogram

kde_trace = go.Scatter(
    x=x_vals,
    y=kde_vals,
    mode='lines',
    name='KDE',
    line=dict(color='darkblue')
)

# Vertical reference lines
vline1 = go.Scatter(
    x=[2000, 2000],
    y=[0, max(kde_vals)],
    mode='lines',
    name='2km Threshold',
    line=dict(color='red', dash='dash')
)

vline2 = go.Scatter(
    x=[2764, 2764],
    y=[0, max(kde_vals)],
    mode='lines',
    name='Median',
    line=dict(color='green', dash='dash')
)

# Create the figure
fig = go.Figure(data=[hist, kde_trace, vline1, vline2])

# Update layout
fig.update_layout(
    title='Distance to CBD at End of Trip',
    xaxis_title='distance_to_cbd_m',
    yaxis_title='Count',
    width=800,
    height=500,
    legend=dict(x=0.7, y=0.95)
)

fig.show( config={'staticPlot':True})


In [ ]:
"""
i will choose the median beacause looking at the histogram we can see the counts drops
"""
threshold = 2764
# Apply binary classification
trips_df['close_to_cbd'] = trips_df['distance_to_cbd_m'].apply(
    lambda d: None if pd.isna(d) else d <= threshold
)
trips_df['close_to_cbd'].value_counts()

In [ ]:
print(trips_df['close_to_cbd'].isna().sum())

---
B8
---

تجميع المواقع الجغرافية باستخدام الهاش الجغرافي (Geohashing)



In [ ]:
"""
Washington, D.C. is roughly:

~16 km (north-south)

~13 km (east-west)

So, a geohash precision of 5–8 is appropriate.
"""
def encode_geohashes(df, lat_col, lon_col, precisions):
    for p in precisions:
        col_name = f'geohash_p{p}'
        df[col_name] = df.apply(lambda row: geohash2.encode(row[lat_col], row[lon_col], p), axis=1)
    return df

# Try precisions from 5 to 8
precisions_to_test = [5, 6, 7, 8]
trips_df = encode_geohashes(trips_df, 'start_lat', 'start_lng', precisions_to_test)
for p in precisions_to_test:
    print(f"Precision {p}: {trips_df[f'geohash_p{p}'].nunique()} unique regions")
"""
If the number is too small → you're over-aggregating.

If it's too big (e.g. thousands) → too fine → hard to summarize meaningfully.
"""

for p in precisions_to_test:
    counts = trips_df[f'geohash_p{p}'].value_counts()
    print(f"Precision {p} → median trips per geohash: {counts.median()}")
"""
This tells us how balanced the spatial bins are.

we ideally want 50–500 trips per cell.
"""

| Precision | Median Trips per Geohash | Interpretation                                                     |
| --------- | ------------------------ | ------------------------------------------------------------------ |
| **5**     | 1761                     | ⚠️ Too coarse — merges many neighborhoods into one.                |
| **6**     | 196                      | ✅ Good balance — each area has enough trips for reliable analysis. |
| **7**     | 7                        | ⚠️ Very fine — may be too sparse for most practical summaries.     |
| **8**     | 2                        | 🚫 Too sparse — most areas will be noise or empty.                 |


In [ ]:
# we will choose 6t
trips_df['geohash_sector'] = trips_df['geohash_p6']


---
B9
---
تقسيم القطاعات الجغرافية (Geohash Sectors) بناءً على حجم الرحلات اليومية

In [ ]:
# Group by Sector and Date
# Assume you have a 'date' column (convert if needed)
trips_df['date'] = pd.to_datetime(trips_df['date'])

# Count trips per day per sector
daily_counts = trips_df.groupby(['geohash_p6', 'date']).size().reset_index(name='trip_count')

# Now compute average daily trips per geohash sector
avg_daily_trips = daily_counts.groupby('geohash_p6')['trip_count'].mean().reset_index()
avg_daily_trips.rename(columns={'trip_count': 'avg_daily_trips'}, inplace=True)


Choose Segmentation Method (for Red / Yellow / Gray)


| Method                         | Description                          | Pros             | Use Case             |
| ------------------------------ | ------------------------------------ | ---------------- | -------------------- |
| **Quantiles** (e.g., tertiles) | Divide into 3 equal-sized groups     | Simple, fair     | Balanced datasets    |
| **KMeans Clustering (k=3)**    | Machine learning-based segmentation  | Optimal grouping | Large datasets       |


In [ ]:
# quantiles :
# Assign labels based on quantiles
quantiles = avg_daily_trips['avg_daily_trips'].quantile([1/3, 2/3])
low_thresh = quantiles.iloc[0]
high_thresh = quantiles.iloc[1]

def classify_volume(val):
    if val < low_thresh:
        return 'gray'   # Low volume
    elif val < high_thresh:
        return 'yellow' # Medium volume
    else:
        return 'red'    # High volume

avg_daily_trips['volume_segment'] = avg_daily_trips['avg_daily_trips'].apply(classify_volume)


In [ ]:

# Extract the data
data = avg_daily_trips['avg_daily_trips'].dropna()

# Histogram trace
hist = go.Histogram(
    x=data,
    nbinsx=30,
    marker_color='lightblue',
    opacity=0.75,
    name='Avg Daily Trips'
)

# Vertical threshold lines
vline_low = go.Scatter(
    x=[low_thresh, low_thresh],
    y=[0, data.value_counts().max()],
    mode='lines',
    name='Low Threshold',
    line=dict(color='gray', dash='dash')
)

vline_high = go.Scatter(
    x=[high_thresh, high_thresh],
    y=[0, data.value_counts().max()],
    mode='lines',
    name='High Threshold',
    line=dict(color='orange', dash='dash')
)

# Combine into figure
fig = go.Figure(data=[hist, vline_low, vline_high])

# Update layout
fig.update_layout(
    title='Distribution of Avg Daily Trips per Geohash Sector',
    xaxis_title='Avg Daily Trips',
    yaxis_title='Count',
    width=800,
    height=500,
    bargap=0.1
)

fig.show(config={'staticPlot':True})


In [ ]:
X = avg_daily_trips[['avg_daily_trips']].values

kmeans = KMeans(n_clusters=3, random_state=42  , n_init=10).fit(X)
avg_daily_trips['kmeans_label'] = kmeans.labels_

# Map to red/yellow/gray using sorted cluster means
label_map = dict(zip(
    np.argsort(kmeans.cluster_centers_.flatten()),
    ['gray', 'yellow', 'red']
))
avg_daily_trips['kmeans_segment'] = avg_daily_trips['kmeans_label'].map(label_map)


In [ ]:
avg_daily_trips.head()

In [ ]:
trips_df['geohash_p6'].nunique()

In [ ]:
# Merge segments into trips_df
trips_df = trips_df.merge(
    avg_daily_trips[['geohash_p6','volume_segment','kmeans_segment']],
    on='geohash_p6',
    how='left'
)


In [ ]:
trips_df['kmeans_segment'].value_counts()


In [ ]:
trips_df['volume_segment'].value_counts()



---
B10
----
تصنيف الظروف الجوية إلى فئات

In [ ]:
trips_df['conditions'].value_counts()

In [ ]:
def classify_weather(condition):
    condition = condition.lower()  # lowercase for safety
    if 'rain' in condition or 'snow' in condition:
        return 'rainy'
    elif 'overcast' in condition or 'cloudy' in condition:
        return 'cloudy'
    elif 'clear' in condition:
        return 'sunny'
    else:
        return 'unknown'

# Apply binning
trips_df['weather_segment'] = trips_df['conditions'].apply(classify_weather)
trips_df['weather_segment'].value_counts()

---
B11
---
إنشاء سلاسل زمنية للإيرادات اليومية حسب الظروف الجوية

quick inspection of the data dates

In [ ]:
sorted_ended_at_df = trips_df[['ended_at']].sort_values(by='ended_at')
print("--- Sorted 'ended_at' DataFrame (first 5 rows) ---")
print(sorted_ended_at_df.head())
print("\n")

# --- Step 3: Find the earliest and latest dates ---
earliest_date = sorted_ended_at_df['ended_at'].min()
latest_date = sorted_ended_at_df['ended_at'].max()

print(f"The earliest date in 'ended_at' is: {earliest_date}")
print(f"The latest date in 'ended_at' is: {latest_date}")


In [ ]:

# --- Step 2: Sort the DataFrame by 'started_at' ---
sorted_started_at_df = trips_df[['started_at']].sort_values(by='started_at')
print("--- Sorted 'started_at' DataFrame (first 5 rows) ---")
print(sorted_started_at_df.head())
print("\n")

# --- Step 3: Find the earliest and latest dates using 'started_at' ---
earliest_date_started = sorted_started_at_df['started_at'].min()
latest_date_started = sorted_started_at_df['started_at'].max()

print(f"The earliest date in 'started_at' is: {earliest_date_started}")
print(f"The latest date in 'started_at' is: {latest_date_started}")


In [ ]:
# Make sure 'ended_at' is datetime
# trips_df['ended_at'] = pd.to_datetime(trips_df['ended_at'])
trips_df['ended_at'] = pd.to_datetime(trips_df['ended_at'], format='mixed', errors='coerce')


# Extract just the date (without time)
trips_df['end_date'] = trips_df['ended_at'].dt.date
daily_income_weather = trips_df.groupby(['end_date', 'weather_segment'])['trip_cost'].sum().reset_index()



In [ ]:
# convert
# Make sure end_date is datetime
daily_income_weather['end_date'] = pd.to_datetime(daily_income_weather['end_date'])

fig_long = px.line(
    daily_income_weather,
    x='end_date',
    y='trip_cost',
    color='weather_segment',
    title='Daily Total Trip Cost by Weather Condition (Long Format)',
    labels={'end_date': 'Date', 'trip_cost': 'Total Income', 'weather_segment': 'Weather'}
)

fig_long.update_layout(xaxis_title='Date', yaxis_title='Trip Cost', hovermode='x unified')
fig_long.show(config={'staticPlot':True})


In [ ]:
# Pivot to wide format
wide_df = daily_income_weather.pivot(index='end_date', columns='weather_segment', values='trip_cost').fillna(0)
wide_df = wide_df.sort_index()

# Build traces
fig_wide = go.Figure()

for condition in wide_df.columns:
    fig_wide.add_trace(go.Scatter(
        x=wide_df.index,
        y=wide_df[condition],
        mode='lines',
        name=condition
    ))

fig_wide.update_layout(
    title='Daily Total Trip Cost by Weather Condition (Wide Format)',
    xaxis_title='Date',
    yaxis_title='Trip Cost',
    hovermode='x unified',
    template='plotly_white',
    legend_title='Weather'
)

fig_wide.show(config={'staticPlot':True})


Which one is better for our problem  ?<br>
the Long Format is the most suitable and effective,This is because it allows for direct visual comparison of revenue trends across different weather types over time on a single graph, making it easier to spot patterns and seasonal impacts. The long format is also considered more intuitive for time-series visualization. Conversely, the "wide format" is deemed less clear due to potential visual clutter when many categories are present.


---
B12
---
إنشاء سمات مساعدة للتحليل الاستكشافي للبيانات (EDA)

Feature 1 : rush_hour
<br> Indicates if the ride occurred during typical commuting hours (7–10 AM or 4–7 PM).

In [ ]:

trips_df['start_time'] = pd.to_datetime(trips_df['start_time'], errors='coerce')

trips_df['rush_hour'] = (
    trips_df['start_time'].dt.hour.between(7, 10) |
    trips_df['start_time'].dt.hour.between(16, 19)
).astype(int)
trips_df['rush_hour'].value_counts()


Feature 2 : hour_segment <br>
Categorize ride start times into broader buckets.

In [ ]:
def get_hour_segment(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Midday'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'

trips_df['hour_segment'] = trips_df['start_time'].dt.hour.apply(get_hour_segment)
trips_df['hour_segment'].value_counts()


Feature 3 : is_weekend<br>
Helps spot usage patterns on weekends vs weekdays.

In [ ]:
trips_df['is_weekend'] = trips_df['start_time'].dt.dayofweek >= 5
trips_df['is_weekend'] = trips_df['is_weekend'].astype(int)
trips_df['is_weekend'].value_counts()



---


#**EDA**


---





---
Sampling the data
---



In [ ]:

# Sampled data stats
sampled_df = trips_df.sample(n=20000, random_state=50)





---

# A )


---





---

# B)

---





---
Task 1
---
رسم مخطط Histogram لتوزيع مدة الرحلة بالدقائق



| Method                     | Formula                         | Notes                               |
| -------------------------- | ------------------------------- | ----------------------------------- |
| **Sturges’ Rule**          | `bins = ceil(log2(n) + 1)`      | Good for small to medium-sized data |
| **Freedman–Diaconis Rule** | `bin_width = 2 * IQR / n^(1/3)` | Good for skewed data or outliers    |
| **Square Root Rule**       | `bins = sqrt(n)`                | Simple and often a good baseline    |


In [ ]:
# Use the sampled dataframe to avoid memory issues
durations = sampled_df['trip_duration_minutes']

# Freedman–Diaconis rule for bin width
q25, q75 = np.percentile(durations, [25, 75])
iqr = q75 - q25
n = len(durations)
bin_width = 2 * iqr / (n ** (1/3))
bin_count = int(np.ceil((durations.max() - durations.min()) / bin_width))

print(f"Suggested bin count: {bin_count}")

# Static histogram
fig = go.Figure(
    data=[go.Histogram(
        x=durations,
        nbinsx=bin_count,
        marker_color='blue',
        opacity=1.0
    )]
)

fig.update_layout(
    title="Distribution of Trip Duration (in Minutes)",
    xaxis_title="Trip Duration (minutes)",
    yaxis_title="Frequency",
    bargap=0.05,
    template='simple_white'
)

fig.show(config={'staticPlot': True})


How many trips took more then a day ?

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Choose your cutoff (in minutes)
cutoff = 1440  # Modify as needed

# Use the sampled dataframe to avoid memory issues
durations = sampled_df['trip_duration_minutes']

# Freedman–Diaconis rule for bin width
q25, q75 = np.percentile(durations, [25, 75])
iqr = q75 - q25
n = len(durations)
bin_width = 2 * iqr / (n ** (1/3))
bin_count = int(np.ceil((durations.max() - durations.min()) / bin_width))

print(f"Suggested bin count: {bin_count}")


# Count how many trips exceed the cutoff
sampled_exceed = (sampled_df['trip_duration_minutes'] > cutoff).sum()
full_exceed = (trips_df['trip_duration_minutes'] > cutoff).sum()

print(f"Trips in sampled_df exceeding {cutoff} minutes: {sampled_exceed}")
print(f"Trips in trips_df exceeding {cutoff} minutes: {full_exceed}")


insights :
1. The massive bar near 0-20 minutes clearly shows that most bike trips are very short. This is typical for bike-sharing systems, often used for short commutes or quick errands.
2. The presence of bars, even if very short, extending all the way to 1440 minutes shoes that some trips in the data did take more then a day , the number of trips is 361





---
Task2
---

رسم مخطط صندوقي  لتوزيع مدة الرحلة حسب نوع الدراجة

In [ ]:
# Use the original (not divided) trip durations
durations = sampled_df['trip_duration_minutes']
types = sampled_df['rideable_type']

# Build the box plot grouped by rideable_type
fig = go.Figure()

# Loop through each rideable type and add a box
for bike_type in sampled_df['rideable_type'].unique():
    fig.add_trace(go.Box(
        y=sampled_df[sampled_df['rideable_type'] == bike_type]['trip_duration_minutes'],
        name=bike_type,
        boxpoints='outliers',  # show outliers only
        marker_color='green',
        line_color='black',
        opacity=0.8
    ))

fig.update_layout(
    title="Box Plot of Trip Duration by Rideable Type",
    yaxis_title="Trip Duration (minutes)",
    xaxis_title="Rideable Type",
    template='simple_white'
)

# Render statically to avoid Colab issues
fig.show(config={'staticPlot': True})


insights :
1. both types show a very compact box near 20 minutes, indicating that the vast majority of trips for both bike types are quite short.
2. The median line is very close to the bottom of the box, confirming heavy right-skewness, this means almost all of the middle 50% of data is concentrated very close to the lower end, and the remaining data (up to Q3) is more spread out.
3. The green dots above the whiskers clearly represent the longer  trips with some of them above the 1440 line
4. Electric bikes seem to have a slightly tighter distribution,This suggests that while both have short typical trips, classic bikes might have a slightly wider range of trips durations
5. Both bike types exhibit very long duration "outliers," with classic bikes potentially having more extreme longer-duration outliers , It suggests that while electric bikes facilitate shorter, perhaps faster trips, classic bikes are used for the most extended journeys.
This could be due to factors like cost , battery limits, or simply user preference .



---
Task3
---
رسم مخطط صندوقي  لتوزيع مدة الرحلة حسب نوع العضوية



In [ ]:
trips_df['member_casual'].value_counts()

In [ ]:
urations = sampled_df['trip_duration_minutes']
types_to_group_by = sampled_df['member_casual']

fig = go.Figure()

for member_type in sampled_df['member_casual'].unique():
    fig.add_trace(go.Box(
        y=sampled_df[sampled_df['member_casual'] == member_type]['trip_duration_minutes'],
        name=member_type,
        boxpoints='outliers',
        marker_color='green',
        line_color='black',
        opacity=0.8
    ))

fig.update_layout(
    title="Box Plot of Trip Duration by Member Type",
    yaxis_title="Trip Duration (minutes)",
    xaxis_title="Member Type",
    template='simple_white'
)

# Render statically to avoid Colab issues
fig.show(config={'staticPlot': True})

insights :
1.	انحراف حاد نحو اليمين لكلا الفئتين: على غرار الرسوم البيانية السابقة لمدة الرحلة، يُظهر مخطط الصندوق لكلا فئتي "المستخدمين العاديين" (casual) و "الأعضاء" (member) انحرافًا قوياً للغاية نحو اليمين, صندوق المخطط لكلتا الفئتين مضغوط للغاية ويقع في الجزء السفلي من الرسم (قريبًا من 0 دقيقة)، ويقع خط الوسيط عملياً فوق الربع الأول (Q1) , هذا يؤكد أن الغالبية العظمى من الرحلات لكلا النوعين من المستخدمين قصيرة جداً.
2. 	مدة رحلة أطول قليلاً للمستخدمين العاديين: على الرغم من أن مدة الرحلات النموذجية قصيرة لكلا الفئتين، إلا أن الوسيط (والربيع الأول/الثالث) للمستخدمين العاديين يبدو أعلى قليلاً (أو على الأقل، الصندوق أقل انضغاطاً بشكل هامشي) من تلك الخاصة بالأعضاء, هذا يشير إلى أن الرحلة "النموذجية" للمستخدم العابر أطول قليلاً من رحلة العضو، حتى لو كانت كلتاهما قصيرتين نسبياً.
3. 	وجود قيم متطرفة للرحلات الطويلة لكلا الفئتين:  تظهر كلتا الفئتين بوضوح عدداً كبيراً من الرحلات "المتطرفة" (النقاط الخضراء) التي تمتد إلى ما هو أبعد بكثير من الصندوق والشعيرات الرئيسية، مما يشير إلى أن الرحلات الطويلة جداً تحدث لكلا النوعين من المستخدمين.
4. 	مدى أوسع للرحلات النموذجية للمستخدمين العاديين: يبدو أن صندوق  )المدى الربيعي (IQR -  للمستخدمين العاديين أوسع بشكل هامشي من ذلك الخاص بالأعضاء, كما أن الشعيرات للمستخدمين العاديين تبدو أطول قليلاً هذا يعني أن المستخدمين العاديين يظهرون مدى أكبر في مدة الرحلات النموذجية مقارنة بالأعضاء.
5.	كثافة أعلى للرحلات المتطرفة الطويلة للمستخدمين العاديين: يمتلك المستخدمون العاديين كثافة أعلى بكثير من الرحلات المتطرفة الطويلة جداً, بينما يمتلك الأعضاء أيضاً رحلات متطرفة طويلة، إلا أنها أقل عدداً وبشكل عام لا تصل إلى نفس المدد القصوى التي يصل إليها المستخدمون العاديين, هذا يشير إلى أن المستخدمين العاديين قد يستخدمون الدراجات لرحلات أطول بشكل متقطع، ربما لأغراض ترفيهية أو سياحية، في حين أن الأعضاء يفضلون الرحلات القصيرة والمنتظمة التي تتناسب مع الاشتراكات.






---
Task4
---
تعيين الرحلات التي تجاوزا يوم كامل



In [ ]:
# Counting Trips Longer Than One Day
# Define threshold: 1 day = 1440 minutes
one_day_minutes = 1440
# Filter trips longer than 1 day
long_trips_df = trips_df[trips_df['trip_duration_minutes'] > one_day_minutes]
long_sampled_df = sampled_df[sampled_df['trip_duration_minutes'] > one_day_minutes]
# Show how many there are
print(f"Total number of trips longer than 1 day in full data: {len(long_trips_df)}")
print(f"Total number of trips longer than 1 day in sampled data: {len(long_sampled_df)}")
# Combine start and end station counts for long trips
start_counts = long_trips_df['start_station_id'].value_counts()
end_counts = long_trips_df['end_station_id'].value_counts()

# Combine them into a single Series
total_counts = start_counts.add(end_counts, fill_value=0).astype(int)

# Get station info: name and location
stations = sampled_df[['start_station_id', 'start_station_name', 'start_lat', 'start_lng']].drop_duplicates()
stations = stations.rename(columns={
    'start_station_id': 'station_id',
    'start_station_name': 'station_name',
    'start_lat': 'lat',
    'start_lng': 'lng'
})

# Merge with counts
stations['long_trip_count'] = stations['station_id'].map(total_counts).fillna(0).astype(int)

# Filter stations with at least 1 long trip
stations = stations[stations['long_trip_count'] > 0]


In [ ]:
# Center the map on Washington DC
m = folium.Map(location=[38.9072, -77.0369], zoom_start=12, tiles='cartodbpositron')

# Optional: cluster points
marker_cluster = MarkerCluster().add_to(m)

# Add stations to the map
for _, row in stations.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lng']],
        radius=3 + row['long_trip_count']**0.5,  # scale marker size
        color='darkred',
        fill=True,
        fill_color='crimson',
        fill_opacity=0.7,
        popup=f"{row['station_name']}<br>Trips > 1 day: {row['long_trip_count']}"
    ).add_to(marker_cluster)

# Show the map
m


insights :
1.  primary observation is that stations associated with long-duration trips are particularly concentrated around the central part WDC map ,This suggests that while very long trips are rare overall, they are not uniformly distributed but rather originate from or end in specific zones.
2. The 2496 is the count of unique station IDs that appear as either a start_station_id OR an end_station_id within those 361 trips.
This is a significant number of stations involved, considering that the total number of such trips was only 361. This means these long trips are spread across a wide variety of stations, rather than being concentrated at just a few specific locations.
3. This highlights areas where the system might need to adapt to different user behaviors like offering specific "long-term rental" options or different pricing for these stations.




---

# C)

---





---

# D)

---





---
Task1
---
عرض مخطط حراري جغرافي لعدد الرحلات



In [ ]:


# --- (Your existing data preparation for res_points) ---
# Assuming 'sampled_df' and 'Residential_Visitor_Parking_Zones' are loaded

# Step 0: Load residential zones GeoDataFrame
res_zones = Residential_Visitor_Parking_Zones
res_zones = res_zones.to_crs(epsg=4326)

# Step 1: Create GeoDataFrames for start and end points (using sampled_df)
start_gdf = gpd.GeoDataFrame(
    sampled_df,
    geometry=gpd.points_from_xy(sampled_df['start_lng'], sampled_df['start_lat']),
    crs='EPSG:4326'
)
end_gdf = gpd.GeoDataFrame(
    sampled_df,
    geometry=gpd.points_from_xy(sampled_df['end_lng'], sampled_df['end_lat']),
    crs='EPSG:4326'
)

# Step 2: Spatial join to check which points fall inside residential zones
start_in_res = gpd.sjoin(start_gdf, res_zones, predicate='within', how='inner')
end_in_res = gpd.sjoin(end_gdf, res_zones, predicate='within', how='inner')

# Step 3: Extract lat/lon of trips touching residential zones
res_start_points = start_in_res[['start_lat', 'start_lng']].rename(columns={'start_lat': 'lat', 'start_lng': 'lon'})
res_end_points = end_in_res[['end_lat', 'end_lng']].rename(columns={'end_lat': 'lat', 'end_lng': 'lon'})
res_points = pd.concat([res_start_points, res_end_points], ignore_index=True)

# --- Load CBD Data and ensure correct CRS (EPSG:4326) ---
CBD = gpd.read_file('Homework/data/DDOT_Central_Business_District.geojson')
CBD = CBD.to_crs(epsg=4326) # Ensure CBD is also in EPSG:4326

# --- Create the Plotly Graph Object Figure ---
fig = go.Figure()

# Add the Heatmap Trace
fig.add_trace(go.Densitymapbox(
    lat=res_points['lat'],
    lon=res_points['lon'],
    radius=10, # Adjust radius as needed for visual effect
    colorscale="Viridis", # Or "Jet", "Hot", "Portland", etc. for heatmap colors
    hoverinfo="skip"
))

# Add Residential Zones as a GeoJSON layer
fig.update_layout(
    mapbox_layers=[
        {
            "below": 'traces',
            "sourcetype": "geojson",
            "source": json.loads(res_zones.to_json()),
            "type": "line",
            # "line": {"width": 1, "color": "blue"} # Corrected: used "color" within "line" object
        },
        # Add CBD as another GeoJSON layer
        {
            "below": 'traces',
            "sourcetype": "geojson",
            "source": json.loads(CBD.to_json()),
            "type": "line",
            # "line": {"width": 2, "color": "green"} # Corrected: used "color" within "line" object
        }
    ]
)

# Update layout for map style, center, zoom, and title
fig.update_layout(
    title_text='Geographic Heatmap of Trips to Residential Zones with Borders',
    mapbox_style="carto-positron",
    mapbox_zoom=10,
    mapbox_center = {"lat": res_points['lat'].mean(), "lon": res_points['lon'].mean()},
    margin={"r":0,"t":40,"l":0,"b":0},
)

# Render statically (Plotly handles this directly when fig.show() is called in a notebook)
fig.show(config={'staticPlot': True})

insights :
1. The most prominent insight is the very high density (bright yellow areas) of bike trips that overwhelmingly coincides with and largely respects the boundaries of the residential zones.
 This indicates that the bike-sharing system is heavily used for purposes directly related to residential areas, whether starting from, ending in, or traversing through them.
2. The highest concentration of trips within residential zones appears to be in the central and northern parts of Washington D.C., This suggests these are the most active residential areas for bike-sharing usage.
3. While the overall pattern is dense,These could indicate areas with fewer stations, different demographic profiles, or less need for bike-sharing, which could be further explored.

---
Task2
---
عرض مخطط شريطي لتوزع فئات القطاعات الجغرافية (Geohash Sectors)


In [ ]:
# Step 1: Count trips per geohash sector
geohash_counts = sampled_df['geohash_p6'].value_counts().reset_index()
geohash_counts.columns = ['geohash_p6', 'trip_count']

# Optional: sort alphabetically or by count
geohash_counts = geohash_counts.sort_values(by='trip_count', ascending=False)

# Step 2: Plot
fig = px.bar(
    geohash_counts,
    x='geohash_p6',
    y='trip_count',
    title='Distribution of Trips by Geographic Sector (Geohash_p6)',
    labels={'geohash_p6': 'Geographic Sector', 'trip_count': 'Number of Trips'}
)

# Step 3: Turn off interactivity
fig.show(config={'staticPlot': True})


In [ ]:


# --- Your Function to Convert Geohash to Polygon ---
def geohash_to_polygon(geohash_str):
    """
    Converts a geohash string to a shapely Polygon representing its bounding box.
    """
    lat, lon, lat_err, lon_err = gh.decode_exactly(geohash_str)
    min_lat = lat - lat_err
    max_lat = lat + lat_err
    min_lon = lon - lon_err
    max_lon = lon + lon_err
    coords = [
        (min_lon, min_lat),
        (max_lon, min_lat),
        (max_lon, max_lat),
        (min_lon, max_lat),
        (min_lon, min_lat)  # Close the polygon
    ]
    return Polygon(coords)

# --- Sample Input: Top 5 Geohashes (replace with your real data) ---
top_5_geohashes = geohash_counts.head(5)['geohash_p6'].tolist()
geohash_polygons = [geohash_to_polygon(g) for g in top_5_geohashes]

# --- Convert to GeoDataFrame for easier plotting ---
gdf = gpd.GeoDataFrame(geometry=geohash_polygons, crs='EPSG:4326')

# --- Plot using Plotly ---
fig = go.Figure()

# Add each polygon as a filled scattermapbox trace
for poly in gdf.geometry:
    lons, lats = poly.exterior.coords.xy
    fig.add_trace(go.Scattermapbox(
        lon=list(lons),
        lat=list(lats),

        mode='lines',
        fill='toself',
        fillcolor='rgba(255, 0, 0, 0.4)',  # Semi-transparent red
        line=dict(width=1, color='red'),
        hoverinfo='skip',
        showlegend=False
    ))

# Map settings
fig.update_layout(
    mapbox_style='carto-positron',
    mapbox_zoom=12,
    mapbox_center={"lat": gdf.geometry.centroid.y.mean(), "lon": gdf.geometry.centroid.x.mean()},
    margin={"r":0,"t":0,"l":0,"b":0}
)

# Static rendering (no interactions)
fig.show(config={'staticPlot': True})


insights :
1.  The first few bars on the left are significantly taller than the rest, with the tallest bar approaching 800 trips, while many others have less than 50. This indicates that bike-sharing activity is heavily concentrated in a few high-demand areas.
2. the plot allows for quick identification of the highest-demand geographic sectors. These are likely to correspond to areas with high population density, major transit hubs, popular attractions, or dense commercial/office districts
3. we ploted these top sectors and we compared it the heatmap in task1 , and we are now sure that they are in hotspots for trips in residential zone




---
Task3
---
عرض مخططات Histogram لتوزيع المسافة عن المنطقة التجارية الرئيسية (CBD)، وأقرب محطة مترو، وأقرب محطة حافلات



In [ ]:

def freedman_diaconis_bins(data):
    data = data.dropna()
    q75, q25 = np.percentile(data, [75 ,25])
    iqr = q75 - q25
    n = len(data)
    if n == 0 or iqr == 0:
        return 10  # fallback if data is empty or IQR is zero
    bin_width = 2 * iqr / (n ** (1/3))
    if bin_width == 0:
        return 10
    bin_count = int(np.ceil((data.max() - data.min()) / bin_width))
    print("Bin count : ", bin_count)
    return bin_count


# Calculate bins for each variable
bins_cbd = freedman_diaconis_bins(sampled_df['distance_to_cbd_m'])
bins_metro = freedman_diaconis_bins(sampled_df['start_nearest_metro_distance'])
bins_shuttle = freedman_diaconis_bins(sampled_df['start_nearest_shuttle_distance'])

sampled_df['distance_to_cbd_km'] = sampled_df['distance_to_cbd_m'] / 1000

# 1. Distance to CBD
fig1 = px.histogram(
    sampled_df,
    x='distance_to_cbd_km',
    nbins=bins_cbd,
    title='Distribution of Distance to CBD (km)',
    labels={'distance_to_cbd_km': 'Distance to CBD (km)'}
)
fig1.show(config={'staticPlot': True})

# 2. Closest Metro Station Distance
fig2 = px.histogram(
    sampled_df,
    x='start_nearest_metro_distance',
    nbins=bins_metro,
    title='Distribution of Distance to Nearest Metro Station',
    labels={'start_nearest_metro_distance': 'Distance to Metro (km)'}
)
fig2.show(config={'staticPlot': True})

# 3. Closest Shuttle Station Distance
fig3 = px.histogram(
    sampled_df,
    x='start_nearest_shuttle_distance',
    nbins=bins_shuttle,
    title='Distribution of Distance to Nearest Shuttle Station',
    labels={'start_nearest_shuttle_distance': 'Distance to Shuttle (km)'}
)
fig3.show(config={'staticPlot': True})


insights :
1. The overwhelming majority of trips (as indicated by the highest bars) occur within approximately 0 to 4 kilometers of the Central Business District. The peak frequency appears to be around 2.2 kilometers, This reinforces the CBD's role as a major hub for bike-sharing activity.
2. this strongly supports the idea that the bike-sharing system primarily serves "last-mile" transportation, short-distance commutes, or intra-CBD movement.


---


1. The histogram displays a highly skewed distribution, with an extremely high frequency of trips originating or ending very close to Metro stations. The counts drop off sharply as the distance increases.
2. This insight highlights the importance of placing bike-sharing stations in very close proximity to Metro entrances to maximize their integration with the public transit network and serve commuter needs effectively.


---
1.  Similar to the Metro station plot, this histogram shows a highly skewed distribution, with the highest frequency of trips occurring very close to shuttle stations.
2. This suggests a consistent user behavior of using bike-sharing to bridge short gaps to fixed public transport points.




---
Task4
---
عرض مخطط شريطي لتوزيع الرحلات بناءً على ما إذا كانت داخل المنطقة التجارية المركزية (CBD) بالكامل أم خارجها



In [ ]:

# Categorize trips
def classify_trip(row):
    if row['start_in_cbd'] == 1 and row['end_in_cbd'] == 1:
        return 'Fully in CBD'
    else:
        return 'Outside CBD'

# Apply classification
sampled_df['cbd_trip_type'] = sampled_df.apply(classify_trip, axis=1)

# Count
trip_cbd_counts = sampled_df['cbd_trip_type'].value_counts().reset_index()
trip_cbd_counts.columns = ['Trip Type', 'Count']

# Plot
fig = px.bar(
    trip_cbd_counts,
    x='Trip Type',
    y='Count',
    title='Trips Fully in CBD vs Outside',
    text='Count',
    labels={'Count': 'Number of Trips'}
)

fig.update_traces(textposition='outside')
fig.update_layout(yaxis_title='Number of Trips', xaxis_title='Trip Category')
fig.show(config={'staticPlot': True})


In [ ]:
trips_df['cbd_trip_type'] = trips_df.apply(classify_trip, axis=1)
full_trip_cbd_counts = trips_df['cbd_trip_type'].value_counts().reset_index()
full_trip_cbd_counts.columns = ['Trip Type', 'Count']
full_trip_cbd_counts['Percentage'] = (full_trip_cbd_counts['Count'] / full_trip_cbd_counts['Count'].sum()) * 100
full_trip_cbd_counts

insights :
1. The vast majority of trips  fall into the "Outside CBD" category, This indicates that the bike-sharing system primarily serves a broader geographical area beyond just the core CBD, or facilitates connections into/out of it rather than exclusively within it.
3.  A significantly smaller number of trips (1,243 trips) were classified as "Fully in CBD". This suggests that while there is some intra-CBD movement, it represents a much smaller proportion
4. This distribution implies that the bike-sharing service's primary function might not be solely for quick hops within the CBD itself, but rather for facilitating commutes, errands, or leisure travel that connects to or traverses the CBD from other parts of the city. This aligns with the "first/last mile" insights from the distance histograms, where trips are often connecting to major transit points or residential areas that might lie outside the strict CBD boundaries.
5.  The fact that over 93% of all the trips involve areas outside the CBD strongly reinforces the idea that bike-sharing is primarily used as a "first/last mile" solution, connecting users to or from transit hubs (like Metro and Shuttle stations) or residential areas that often lie outside the strict CBD boundaries. These trips are unlikely to be exclusively within the CBD.



---
Task5
---
•	عرض مخطط شريطي لتوزيع الرحلات التي مرت عبر المنطقة التجارية المركزية (CBD) حسب نوع الدراجة ونوع العضوية


In [ ]:

# Filter trips that passed through CBD
cbd_passed_df = sampled_df[
    (sampled_df['start_in_cbd'] == 1) | (sampled_df['end_in_cbd'] == 1)
]

# Group by rideable_type and member_casual
grouped = cbd_passed_df.groupby(['rideable_type', 'member_casual']).size().reset_index(name='trip_count')

# Plot
fig = px.bar(
    grouped,
    x='rideable_type',
    y='trip_count',
    color='member_casual',
    barmode='group',
    title='Trips That Passed Through CBD by Rideable Type and Membership',
    labels={'trip_count': 'Number of Trips', 'rideable_type': 'Bike Type'}
)

fig.update_layout(
    xaxis_title='Rideable Type',
    yaxis_title='Number of Trips'
)
fig.show(config={'staticPlot': True})


In [ ]:
cbd_passed_df_trips_df=trips_df[
    (trips_df['start_in_cbd'] == 1) | (trips_df['end_in_cbd'] == 1)
]

# Group by rideable_type and member_casual
grouped = cbd_passed_df_trips_df.groupby(['rideable_type', 'member_casual']).size().reset_index(name='trip_count')

print(f"Length of cbd_passed_df_trips_df: {len(cbd_passed_df_trips_df)}")
print(f"Length of trips_df: {len(trips_df)}")

percentage = (len(cbd_passed_df_trips_df) / len(trips_df)) * 100
print(f"Percentage of cbd_passed_df_trips_df compared to trips_df: {percentage:.2f}%")

insights :
1. the extra exploration shows that 29.85% of all bike-sharing trips pass through the CBD, This is a substantial portion, indicating the CBD's critical role as an origin, destination, or transit point for a large segment of bike-sharing users.
2. For both classic_bike and electric_bike categories, members (orange/red bars) consistently account for a significantly higher number of trips that pass through the CBD compared to casual riders (blue bars). This suggests that regular commuters or frequent users (members) are more likely to utilize bike-sharing for travel involving the city's central business district.
3. Both casual and member riders use classic_bike more frequently for CBD-involved trips than electric_bike
4. Since members are the primary users for CBD-involved trips, strategies to retain and grow membership, and ensure sufficient bike availability in and around the CBD, are crucial.
5. The demand for both classic and electric bikes for CBD-involved trips means that fleet management should consider a balanced distribution to meet the preferences of both member and casual riders.




---
Task6
---
دراسة الارتباط بين فئة البعد عن المنطقة التجارية الرئيسية (CBD) ونوع العضوية باستخدام تحليل  Chi-Square



In [ ]:
# Create a contingency table
# (Counts of each combination)
# Make sure we’re using categorical data
contingency_table = pd.crosstab(trips_df['close_to_cbd'], trips_df['member_casual'])
contingency_table


In [ ]:
# Run chi-square test
chi2, p, dof, expected = chi2_contingency(contingency_table)
print("Chi2 Statistic:", chi2)
print("Degrees of Freedom:", dof)
print("P-value:", p)
# interpretion based on the p-value:
if p < 0.05:
    print(" There is a significant correlation between distance to CBD segments and membership type.")
else:
    print(" No significant correlation found between distance to CBD segments and membership type.")


| α Value  | Interpretation                                                                |
| -------- | ----------------------------------------------------------------------------- |
| **0.05** | Most common — means you're willing to accept a 5% chance of a false positive. |
| 0.01     | Stricter — used in more critical fields (medicine, etc.).                     |
| 0.10     | Looser — sometimes used in exploratory research.                              |


insights:
1. The p-value of 0.0, which is much less than the conventional significance level of 0.05 leads us to believe there is a statistically significant relationship between whether a trip is close to the CBD and the user's membership type.
2. While the Chi-square test indicates strong statistical significance (due to the very large sample size), the proportional difference between casual and member trips being close_to_cbd is relatively small,This suggests thatBoth member and casual riders have roughly half their trips originating or ending close to the CBD.

# Secret mission




---

**Knowledge Discovery & Storytelling**

---





---

1. How do peak hours and days (rush hour, day of week, weekend) vary for different rideable_type and member_casual groups?
<br>
Answer here :
2. Are there distinct usage patterns for electric_bike vs. classic_bike during different hour_segment and start_day_name, and does this differ between member_casual types?
<br>
Answer here :
3. How do specific weather_segment impact trip_duration_minutes and trip_cost for different rideable_type?
<br>
Answer here :
4. Do start_station_name and end_station_name popularity, or overall trip density (geohash_sector), show significant differences between is_weekend and weekdays?
<br>
Answer here :
5. Which stations act as major transfer hubs (high start_station_name and end_station_name volume)? Do they primarily serve specific member_casual types or rideable_type?
<br>
Answer here :
6. Can we identify common "commute routes" or "regular paths" for users based on frequent start_station_name and end_station_name pairs?
<br>
Answer here :
7. Given the April revenue issue, can we pinpoint a potential cause by analyzing weather_segment, changes in member_casual behavior, or rideable_type availability/usage during that specific month?
<br>
Answer here :
8. Is there any correlation between a station's volume_segment and its performance during rush_hour or is_weekend, and does this performance vary by weather_segment?
<br>
Answer here :



---



In [ ]:
trips_df.info()



---
َQ1
---
How do peak hours and days (rush hour, day of week, weekend) vary for different rideable_type and member_casual groups?


In [ ]:
def aggregate_trip_counts(df, group_col):
    return (
        df
        .groupby(['member_casual', 'rideable_type', group_col])
        .size()
        .reset_index(name='ride_count')
    )

def plot_static_trip_bar(df, x_col, title, x_label):
    fig = px.bar(
        df,
        x=x_col,
        y='ride_count',
        color='rideable_type',
        barmode='group',
        facet_col='member_casual',
        title=title,
        labels={x_col: x_label, 'ride_count': 'Number of Rides'}
    )
    fig.update_layout(showlegend=True)
    fig.show(config={'staticPlot': True})  # Static rendering


In [ ]:
# for rush hour
(
    trips_df
    .pipe(aggregate_trip_counts, group_col='rush_hour')
    .pipe(plot_static_trip_bar, x_col='rush_hour', title='Rush Hour Usage by Type & Member', x_label='Rush Hour (1=True, 0=False)')
)


____________
casual  
1. Significantly fewer rides during rush hour
2.  during non-rush hour, electric bikes are slightly preferred over classic bikes.

members
1. Substantially higher number of rides during non-rush hour
2. Electric bikes are also preferred during non-rush hour.
_______

In [ ]:
#  for start day name
trips_df['start_day_name'] = pd.Categorical(
    trips_df['start_day_name'],
    categories=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
    ordered=True
)

(
    trips_df
    .pipe(aggregate_trip_counts, group_col='start_day_name')
    .sort_values('start_day_name')
    .pipe(plot_static_trip_bar, x_col='start_day_name', title='daily Usage Patterns by Type & Member', x_label='Day of the Week')
)


____________
casual  
1. Usage gradually increases throughout the weekdays, peaking on weekends (saturday and sunday), and then slightly higher on weekends (Saturday being highest).
2.  Electric bikes are consistently more popular than classic bikes across all days of the week.

members
1. Usage is highest during weekdays (Monday to Friday), indicating regular commuting patterns.
2. Usage drops significantly on weekends (Saturday and Sunday), which is the opposite of the casual.
3. Electric bikes are consistently and significantly more popular than classic ,On weekends, electric bike usage remains higher but the gap between electric and classic bikes narrows compared to weekdays.


_______

In [ ]:
# for  is weekend
(
    trips_df
    .pipe(aggregate_trip_counts, group_col='is_weekend')
    .pipe(plot_static_trip_bar, x_col='is_weekend', title='Weekend vs Weekday Rides by Type & Member', x_label='Is Weekend (1=True, 0=False)')
)


____________
casual  
1. Electric bikes are more popular than classic bikes for both weekdays and weekends, but especially during weekdays.


members
1. On weekends, classic bike usage kinda equal electric bike usage
_______

**Final Answer :**

تختلف أوقات الذروة وأيام الاستخدام بشكل كبير بين فئات المستخدمين وأنواع الدراجات, يظهر الأعضاء أنماط استخدام قوية في أيام الأسبوع وساعات غير الذروة، ويفضلون الدراجات الكهربائية بشكل عام، باستثناء ساعات الذروة وعطلات نهاية الأسبوع حيث يكون هناك استخدام ملحوظ للدراجات الكلاسيكية. في المقابل، يميل المستخدمون العابرون إلى زيادة استخدامهم نحو نهاية الأسبوع ويفضلون الدراجات الكهربائية.



---
َQ2
---
 Are there distinct usage patterns for electric_bike vs. classic_bike during different hour_segment and start_day_name, and does this differ between member_casual types?


In [ ]:
# for hour segments
(
    trips_df
    .pipe(aggregate_trip_counts, group_col='hour_segment')
    .pipe(plot_static_trip_bar, x_col='hour_segment', title='Usage by Hour Segment and Ride Type', x_label='Hour Segment')
)


________
1. Electric Bike Dominant in night time while classis is slightly more in the the evening

2. both casual and members usage peakes at night time segment

**Final Answer :**<br>
Yes, there are distinct usage patterns for electric vs. classic bikes, and these patterns vary between member and casual user types across different hourly segments and days of the week. Electric bikes show a general and dominant preference across both user types  during the "Night" segment
what does this tells us ?
it could be the following
1. Late-Night Commuting/Work Shifts
2.  During the "Night" segment, public transportation options (buses, metro) often become less frequent or stop running, making electric bikes a convenient and perhaps faster alternative for members needing to travel.
3. Convenience and Effort Reduction
<br>


---
Q3
---
How do specific weather_segment or conditions (e.g., "Light Rain," "Clear") impact trip_duration_minutes and trip_cost for different rideable_type?

In [ ]:
def aggregate_weather_impact(df, group_col):
    return (
        df
        .groupby(['rideable_type', group_col])
        .agg(
            avg_duration=('trip_duration_minutes', 'mean'),
            median_duration=('trip_duration_minutes', 'median'),
            avg_cost=('trip_cost', 'mean'),
            median_cost=('trip_cost', 'median'),
            trip_count=('trip_duration_minutes', 'count')
        )
        .reset_index()
    )

def plot_avg_duration(df, x_col, title):
    fig = px.bar(
        df,
        x=x_col,
        y='avg_duration',
        color='rideable_type',
        barmode='group',
        title=title,
        labels={'avg_duration': 'Avg Trip Duration (min)', x_col: x_col}
    )
    fig.update_layout(showlegend=True)
    fig.show(config={'staticPlot': True})

# average trip cost
def plot_avg_cost(df, x_col, title):
    fig = px.bar(
        df,
        x=x_col,
        y='avg_cost',
        color='rideable_type',
        barmode='group',
        title=title,
        labels={'avg_cost': 'Avg Trip Cost ($)', x_col: x_col}
    )
    fig.update_layout(showlegend=True)
    fig.show(config={'staticPlot': True})
# Duration by weather segment
(
    trips_df
    .pipe(aggregate_weather_impact, group_col='weather_segment')
    .pipe(plot_avg_duration, x_col='weather_segment', title='Avg Trip Duration by Weather Segment and Ride Type')
)

# Cost by weather segment
(
    trips_df
    .pipe(aggregate_weather_impact, group_col='weather_segment')
    .pipe(plot_avg_cost, x_col='weather_segment', title='Avg Trip Cost by Weather Segment and Ride Type')
)


1.  Across all weather conditions (Cloudy, Rainy, Sunny), classic bikes consistently show a significantly higher average trip duration compared to electric bikes.
2. For classic bikes, the average duration is slightly higher on sunny and cloudy days (around 17-18 minutes), and decreases slightly on rainy days (just over 15 minutes), suggesting users might shorten their rides slightly in the rain.
3. For electric bikes, the average trip duration remains relatively stable and around 12-13 minutes across all weather conditions, indicating their use for quick and efficient trips regardless of the weather.
4. both bike types avg trip cost is stable in every condition , this incdicates general weather condition  has a very minimal, almost negligible, impact on the average trip cost for both types of bikes.


**Final Answer :**<br>
Weather conditions have a small impact on the average trip duration and average trip cost,  Classic bikes consistently record a longer average trip duration than electric bikes across all weather conditions, with a slight decrease on rainy days. In contrast, electric bikes are used for shorter average trips and have a consistently slightly higher average cost, but their duration and cost are not  affected by changes in general weather conditions. This suggests that the primary factors influencing trip duration and cost are related to the bike type itself rather than the overarching weather segments.

---
Q4
---
Do start_station_name and end_station_name popularity, or overall trip density (geohash_sector), show significant differences between is_weekend and weekdays?


In [ ]:
def aggregate_station_or_zone_avg_per_day(df, group_col, top_n=15):
    grouped = (
        df
        .groupby([group_col, 'is_weekend'])
        .size()
        .reset_index(name='ride_count')
    )

    # Normalize: divide by number of days in each category
    grouped['avg_per_day'] = grouped.apply(
        lambda row: row['ride_count'] / 2 if row['is_weekend'] == 1 else row['ride_count'] / 5,
        axis=1
    )

    # Get top N based on total ride counts (still for filtering)
    top_items = (
        grouped
        .groupby(group_col)['ride_count']
        .sum()
        .nlargest(top_n)
        .index
    )

    return grouped[grouped[group_col].isin(top_items)]
def plot_static_avg_bar(df, x_col, title):
    fig = px.bar(
        df,
        x=x_col,
        y='avg_per_day',
        color='is_weekend',
        barmode='group',
        title=title,
        labels={x_col: x_col.replace("_", " ").title(), 'avg_per_day': 'Avg Rides per Day', 'is_weekend': 'Is Weekend'}
    )
    fig.update_layout(showlegend=True)
    fig.show(config={'staticPlot': True})

(
    trips_df
    .pipe(aggregate_station_or_zone_avg_per_day, group_col='start_station_name', top_n=15)
    .pipe(plot_static_avg_bar, x_col='start_station_name', title='Avg Daily Rides from Top 15 Start Stations (Weekday vs Weekend)')
)

(
    trips_df
    .pipe(aggregate_station_or_zone_avg_per_day, group_col='end_station_name', top_n=15)
    .pipe(plot_static_avg_bar, x_col='end_station_name', title='Avg Daily Rides to Top 15 End Stations (Weekday vs Weekend)')
)



1. Weekday Dominance, For nearly all of the top 15 stations, the vast majority of rides originate on weekdays
2. "Park Rd & Holmead Pl NW" stands out as the most popular starting and end station
3. The consistent weekday dominance across these high-density zones suggests that the core bike-sharing activity in the city's busiest areas is predominantly driven by weekday behaviors, such as commuting or running errands.

**Final answer**<br>
Yes, the popularity of both start_station_name and end_station_name, along with overall trip density as measured by geohash_sector, exhibit significant differences between weekdays and weekends. weekday usage consistently and heavily outweighs weekend usage. This indicates that the primary demand and busiest areas for bike-sharing are concentrated during the workweek, suggesting a strong influence of commuting and routine daily activities. While weekend rides do occur, they form a notably smaller proportion of total activity in these popular hubs

---
Q5
---
Which stations act as major transfer hubs (high start_station_name and end_station_name volume)? Do they primarily serve specific member_casual types or rideable_type?

In [ ]:
def get_top_transfer_stations(df, top_n=15):
    # Count start and end occurrences separately
    starts = df['start_station_name'].value_counts()
    ends = df['end_station_name'].value_counts()

    # Sum to get total transfer volume
    total_volume = starts.add(ends, fill_value=0).sort_values(ascending=False)

    top_stations = total_volume.head(top_n).index.tolist()
    return top_stations

def aggregate_station_transfer_profiles(df, station_list):
    filtered = df[
        df['start_station_name'].isin(station_list) |
        df['end_station_name'].isin(station_list)
    ].copy()

    # Normalize to one column: station_name
    filtered['station_name'] = filtered.apply(
        lambda row: row['start_station_name'] if row['start_station_name'] in station_list else row['end_station_name'],
        axis=1
    )

    # Group by station, member_casual, rideable_type
    return (
        filtered
        .groupby(['station_name', 'member_casual', 'rideable_type'])
        .size()
        .reset_index(name='ride_count')
    )

def plot_station_transfer_breakdown(df, title):
    fig = px.bar(
        df,
        x='station_name',
        y='ride_count',
        color='rideable_type',
        barmode='group',
        facet_col='member_casual',
        title=title,
        labels={
            'station_name': 'Station Name',
            'ride_count': 'Ride Count',
            'rideable_type': 'Bike Type',
            'member_casual': 'User Type'
        }
    )
    fig.update_layout(showlegend=True, xaxis_tickangle=-45)
    fig.show(config={'staticPlot': True})
top_stations = get_top_transfer_stations(trips_df, top_n=15)

(
    trips_df
    .pipe(aggregate_station_transfer_profiles, station_list=top_stations)
    .pipe(plot_station_transfer_breakdown, title='Top 15 Transfer Hubs by User and Bike Type')
)


1. The plot clearly indicates that "Park Rd & Holmead Pl NW" is the most significant transfer hub
2. These major transfer hubs overwhelmingly serve member users
3. For both casual and member users, electric bikes  are consistently more utilized than classic bikes  across almost all of the top 15 transfer stations.


**Final answer** <br>
The station acting as the primary major transfer hub is "Park Rd & Holmead Pl NW", followed by stations such as "14th St & New York Ave NW," "17th & P St NW," and "14th & Belmont St NW". These hubs overwhelmingly serve member users, who exhibit significantly higher ride volumes than casual users at these locations. Furthermore, these transfer hubs primarily cater to the use of electric bikes, which are consistently and notably more popular than classic bikes for both member and casual riders at these high-volume stations.

---
Q6
---
Can we identify common "commute routes" or "regular paths" for  users based on frequent start_station_name and end_station_name pairs?

In [ ]:
def aggregate_top_routes_with_cbd(df, top_n=15):
    # Filter to classic/electric only
    filtered_df = df[df['rideable_type'].isin(['classic_bike', 'electric_bike'])].copy()

    # Create route string
    filtered_df['route'] = filtered_df['start_station_name'] + ' → ' + filtered_df['end_station_name']

    # Group by user, route, and in_cbd
    route_counts = (
        filtered_df
        .groupby(['member_casual', 'route', 'in_cbd'])
        .size()
        .reset_index(name='ride_count')
    )

    # Get top N routes *per user*, regardless of CBD
    top_routes = (
        route_counts
        .groupby('member_casual', group_keys=False)
        .apply(lambda g: g.groupby('route')['ride_count'].sum().nlargest(top_n))
        .reset_index()
        .rename(columns={0: 'total_rides'})
    )

    # Merge back to get `in_cbd` split per route
    final_df = route_counts.merge(top_routes[['route']], on='route', how='inner')

    return final_df
def plot_top_routes_cbd(df, title):
    fig = px.bar(
        df,
        x='ride_count',
        y='route',
        color='in_cbd',
        facet_col='member_casual',
        orientation='h',
        title=title,
        labels={
            'route': 'Route (Start → End)',
            'ride_count': 'Number of Rides',
            'in_cbd': 'Passed CBD',
            'member_casual': 'User Type'
        },
        color_discrete_map={0: '#636EFA', 1: '#EF553B'}
    )
    fig.update_layout(showlegend=True)
    fig.update_yaxes(categoryorder="total ascending")
    fig.show(config={'staticPlot': True})
(
    trips_df
    .pipe(aggregate_top_routes_with_cbd, top_n=15)
    .pipe(plot_top_routes_cbd, title='Top 15 Routes by User Type and CBD Inclusion')
)


1. There are clearly a few routes that are significantly more popular than others,The self-loop routes (start and end at the same station) are very prominent.
2.  For many of the top routes, a significant portion of trips seem to involve the CBD


**Final answer**<br>
Yes, we can identify common "regular paths" for users based on frequent start_station_name and end_station_name pairs. The analysis of the "Top 15 Routes" plot reveals distinct popular paths. While several prominent routes are self-looping , a considerable portion of these top routes also show involvement with the Central Business District (in_cbd). This in_cbd presence suggests that many of these frequent paths, likely serve as actual "commute routes" or regular travel paths for users, especially those traveling to, from, or through the city's commercial core.


---
Q7
---
Given the April revenue issue, can we pinpoint a potential cause by analyzing weather_segment, changes in member_casual behavior, or rideable_type availability/usage during that specific month?

In [ ]:
trips_df['start_month'].value_counts()

In [ ]:
def filter_april(df):
    return df[df['start_month'] == 4].copy()
def aggregate_april_by_user(df):
    return (
        df
        .groupby('member_casual')
        .size()
        .reset_index(name='ride_count')
    )

def aggregate_april_by_bike(df):
    return (
        df
        .groupby('rideable_type')
        .size()
        .reset_index(name='ride_count')
    )

def aggregate_april_by_weather(df):
    return (
        df
        .groupby('weather_segment')
        .size()
        .reset_index(name='ride_count')
    )

def aggregate_april_cost_stats(df):
    return (
        df
        .groupby(['rideable_type', 'member_casual', 'weather_segment'])
        .agg(
            total_rides=('ride_id', 'count'),
            avg_trip_cost=('trip_cost', 'mean'),
            total_revenue=('trip_cost', 'sum')
        )
        .reset_index()
    )
def plot_april_bar(df, x, y, title, color=None):
    fig = px.bar(
        df,
        x=x,
        y=y,
        color=color,
        title=title,
        labels={x: x.replace("_", " ").title(), y: y.replace("_", " ").title()}
    )
    fig.update_layout(showlegend=True)
    fig.show(config={'staticPlot': True})

(
    trips_df
    .pipe(filter_april)
    .pipe(aggregate_april_by_user)
    .pipe(lambda df: plot_april_bar(df, x='member_casual', y='ride_count', title='April Ride Volume by User Type'))
)

(
    trips_df
    .pipe(filter_april)
    .pipe(aggregate_april_by_bike)
    .pipe(lambda df: plot_april_bar(df, x='rideable_type', y='ride_count', title='April Ride Volume by Bike Type'))
)

(
    trips_df
    .pipe(filter_april)
    .pipe(aggregate_april_by_weather)
    .pipe(lambda df: plot_april_bar(df, x='weather_segment', y='ride_count', title='April Ride Volume by Weather Segment'))
)

(
    trips_df
    .pipe(filter_april)
    .pipe(aggregate_april_cost_stats)
)



In [ ]:
tri